In [ ]:
import numpy as np
import pandas as pd
import gtfs_functions
import yaml
from adjustText import adjust_text
import matplotlib as mpl
import contextily as cx
import folium

In [ ]:
mpl.rcParams["figure.figsize"] = [19.2, 14.4] # default [6.4, 4.8]

In [ ]:
feed = gtfs_functions.Feed("gtfs.zip")

In [ ]:
# One of "2", "3", "6", "10", "11", "18", "54", "64", "21", "63"
# Note that 21 and 63 are entirely high frequency
# Source: https://www.metrotransit.org/high-frequency-network
# Map: 
route_id = "64"
stop_ids = feed.stop_times[feed.stop_times.route_id == route_id].stop_id.unique()

In [ ]:
df = feed.stops[feed.stops.stop_id.isin(stop_ids)]
high_frequency_stop_codes = yaml.safe_load(open("high_frequency_stops.yaml"))
df["is_high_frequency"] = df.stop_code.isin(high_frequency_stop_codes)

df_wm = df.to_crs(epsg=3857)
ax = df_wm.plot(column="is_high_frequency", markersize=15)
texts = df_wm.apply(lambda row: ax.annotate(text=row.stop_code, xy=row.geometry.centroid.coords[0], ha="center", fontsize="medium"), axis=1)
adjust_text(list(texts), arrowprops=dict(arrowstyle="->", color="black"))
cx.add_basemap(ax, reset_extent=False)
ax.set_axis_off()

In [ ]:
df = feed.stops[feed.stops.stop_id.isin(stop_ids)]
high_frequency_stop_codes = yaml.safe_load(open("high_frequency_stops.yaml"))
df["is_high_frequency"] = df.stop_code.isin(high_frequency_stop_codes)
df.explore("is_high_frequency", tooltip=["stop_code", "stop_name"])

In [ ]:
np.diff(np.sort(feed.stop_times[(feed.stop_times.route_id == route_id) & (feed.stop_times.stop_id == "")].arrival_time.unique() / 60))

In [ ]:
feed.stops[feed.stops.stop_code.isin(high_frequency_stop_codes)].explore()